In [1]:
import os 
from dotenv import load_dotenv

load_dotenv()

print(os.getenv("AZURE_OPENAI_ENDPOINT"))

database_name = os.getenv('SQL_DATABASE_NAME')

https://openai-sweden-central-andreea.openai.azure.com/


In [2]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    OpenAIChatCompletion,
)

kernel = sk.Kernel()

deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

kernel.add_chat_service(
    "chat_completion",
    AzureChatCompletion(deployment_name=deployment, endpoint=endpoint, api_key=api_key),
)

Kernel(plugins=KernelPluginCollection(plugins={}), prompt_template_engine=PromptTemplateEngine(), memory=NullMemory(), text_completion_services={'chat_completion': <function Kernel.add_text_completion_service.<locals>.<lambda> at 0x000001DE7F59DA80>}, chat_services={'chat_completion': <function Kernel.add_chat_service.<locals>.<lambda> at 0x000001DE7F59D9E0>}, text_embedding_generation_services={}, default_text_completion_service='chat_completion', default_chat_service='chat_completion', default_text_embedding_generation_service=None, retry_mechanism=PassThroughWithoutRetry(), function_invoking_handlers={}, function_invoked_handlers={})

In [3]:
sk_prompt = """
Generate a SQL query for a SQL Server database to retrieve data from an ERP system. Database name is called: {{$database}}
Take into consideration that the schema for the Product table is the following: {{$schema}}
The query should return the required data to satisfy the request of the user: {{$user_request}}"""

In [4]:
chat_function = kernel.create_semantic_function(
    prompt_template=sk_prompt,
    function_name="ChatBot",
    max_tokens=2000,
    temperature=0.4,
    top_p=0.5,
)

In [5]:
schema = """ ProductID 
          int 
          PRIMARY KEY
          Not Nullable        
  
          Name 
          nvarchar 
          No Constraint
          Not Nullable        
  
          ProductNumber 
          nvarchar 
          No Constraint
          Not Nullable"""

context = kernel.create_new_context()
context["database"] = database_name
context["schema"] = schema
context["user_request"] = "I want to see what are the top 5 products in our catalog"

bot_answer = await chat_function.invoke(context=context)

print(bot_answer)


Assuming that "top 5 products" refers to the products with the highest ProductID, the SQL query would be:

```sql
SELECT TOP 5 *
FROM AdventureWorks_v1.dbo.Product
ORDER BY ProductID DESC;
```

However, if "top 5 products" refers to the most sold products, you would need information from other tables such as SalesOrderDetail or similar. In that case, the query might look something like this:

```sql
SELECT TOP 5 p.ProductID, p.Name, p.ProductNumber, COUNT(s.ProductID) as QuantitySold
FROM AdventureWorks_v1.dbo.Product p
JOIN AdventureWorks_v1.dbo.SalesOrderDetail s ON p.ProductID = s.ProductID
GROUP BY p.ProductID, p.Name, p.ProductNumber
ORDER BY QuantitySold DESC;
```

Please note that the second query assumes that there is a table called SalesOrderDetail with a column ProductID that records every sale of each product. The actual table and column names might be different in your database.


Use semantic function from folder

In [ ]:
plugin_func = kernel.import_semantic_plugin_from_directory("plugins/", "GetDataPlugin")

data_plugin = plugin_func["GetData"]

In [9]:
result = await data_plugin.invoke(context=context)

print(result)

SELECT TOP 5 Name, ProductNumber 
FROM [table_name] 
ORDER BY ProductID DESC;
